In [1]:
!pip install py2neo

In [2]:
!pip install elasticsearch

In [3]:
#製作flask環境
from flask import Flask, request, jsonify, session, redirect, url_for, render_template, flash
import pymysql

from elasticsearch import Elasticsearch
from py2neo import Graph, Node, Relationship
import json 
import datetime
import os
import uuid

es = Elasticsearch(['elasticsearch:9200'])

#增加等待時間，為了整合的需要所新增的 
#import time
#time.sleep( 100 )

#呼叫出Flask
app = Flask(__name__)


#建立與mysql的連線
conn = pymysql.connect(host='db', port=3306, user='root', passwd='iii', db='chatbot_db',charset='utf8mb4')

#方便用來跟mysql互動
cur = conn.cursor()

In [4]:
# 存入新聞
@app.route('/news',methods=['POST'])
def add_news():
    
    # 取得新聞資料
    news = request.get_json()
    
    # 定義錯誤資訊
    error = None
    
    # 檢查新聞是否重複
    cur.execute('SELECT title_id FROM chatbot_db.News WHERE url = ("%s")' % (news['url']))
    title_id = cur.fetchone() 
    if not title_id == None :
        error = 'News {} is exist.'.format(news['url'])
    
    
    #若無重複
    if error == None:
    
        # 定義儲存時間
        store_datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # 從DB取出來源id
        cur.execute('SELECT origin_id FROM chatbot_db.Origin WHERE origin = ("%s")' % (news['source']))
        origin_id = cur.fetchone() 
        
        # 確認新聞來源是否存在，若不存在存入新聞來源
        if origin_id == None:
        
            # 存入來源
            insertsql ='INSERT INTO chatbot_db.Origin (origin) VALUES (%s)'
            value = (news['source'])
            cur.execute(insertsql , value)
            conn.commit()
            
            # 取出剛存入的新聞來源
            cur.execute('SELECT origin_id FROM chatbot_db.Origin WHERE origin = ("%s")' % (news['source']))
            origin_id = cur.fetchone() 
        
        # 從DB取出標籤id
        cur.execute('SELECT label_id FROM chatbot_db.Label WHERE label = ("%s")' % (news['label']))
        label_id = cur.fetchone()
        
        # 從DB取出情緒id
        cur.execute('SELECT article_emotion_id FROM chatbot_db.Article_Emotion WHERE ariticle_emotion = ("%s")' % (news['ariticle_emotion']))
        article_emotion_id = cur.fetchone()
        
        # 存入新聞資料
        insertsql= "INSERT INTO chatbot_db.News (origin_id, label_id, article_emotion_id, title,url, release_datetime, content, abstract, img_url, store_datetime) VALUES ( %s,%s,%s,%s,%s,%s,%s,%s,%s,%s )" 
        value =( origin_id, 
                 label_id, 
                 article_emotion_id,
                 news['title'],
                 news['url'],
                 news['date_'],
                 news['content'],
                 news['abstract'],
                 news['img_url'],
                 store_datetime
        )
        
        cur.execute(insertsql , value)
        #將資料送進資料庫中
        conn.commit()
        
        # 找出剛存入的新聞 title_id
        cur.execute('SELECT title_id FROM chatbot_db.News WHERE url = ("%s")' % (news['url']))
        title_id = cur.fetchone()
        
        # 存入記者資料
        if not news['author'] == []:
            author_list = news['author']
            for author in author_list:

                # 檢查記者是否存在資料庫
                cur.execute('SELECT author_id FROM chatbot_db.Author WHERE author = ("%s")' % (author))
                author_id = cur.fetchone()

                # 若不存在則先存入記者並取出 author_id
                if author_id == None:
                    insertsql= "INSERT INTO chatbot_db.Author (author) VALUES (%s)"
                    value = (author)
                    cur.execute(insertsql , value)
                    conn.commit()

                    cur.execute('SELECT author_id FROM chatbot_db.Author WHERE author = ("%s")' % (author))
                    author_id = cur.fetchone()

                # 將記者資訊存入DB
                insertsql= "INSERT INTO chatbot_db.Author_Write (title_id,author_id) VALUES (%s, %s)"
                value = (title_id,author_id)
                cur.execute(insertsql , value)
                conn.commit()

        # 存入 keyword
        if not news['kw'] == []:
        
            keyword_list = news['kw']
            for keyword in keyword_list:

                # 檢查關鍵字是否存在資料庫
                cur.execute('SELECT keyword_id FROM chatbot_db.Keyword WHERE keyword = ("%s")' % (keyword))
                keyword_id = cur.fetchone()

                # 若不存在則先存入關鍵字並取出 keyword_id
                if keyword_id == None:
                    insertsql= "INSERT INTO chatbot_db.Keyword (keyword) VALUES (%s)"
                    value = (keyword)
                    cur.execute(insertsql , value)
                    conn.commit()

                    cur.execute('SELECT keyword_id FROM chatbot_db.Keyword WHERE keyword = ("%s")' % (keyword))
                    keyword_id = cur.fetchone()

                # 將關鍵字資訊存入DB
                insertsql= "INSERT INTO chatbot_db.Article_Keyword (title_id,keyword_id) VALUES (%s, %s)"
                value = (title_id,keyword_id)
                cur.execute(insertsql , value)
                conn.commit()

        #回傳一個正確的描述
        result =  { "status_describe":"success add News"}

    # 若有重複,回傳存在資訊
    else:

         result = {"status_describe":"{}".format(error)}

    return jsonify(result)

In [5]:
# 存入使用者
@app.route('/users', methods = ['POST'])
def add_user():

    # 定義儲存時間
    join_datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    users = request.get_json()

    error = None

    cur.execute('SELECT display_name FROM chatbot_db.Users WHERE user_id = ("%s")' % (users['user_id']))
    display_name = cur.fetchone()

    # 檢查id是否重複
    if not display_name == None:
        # 若重複給予錯誤訊息
        error = 'User {} {} is already registered.'.format(users['user_id'], display_name)

        #製作一個錯誤的描述
        result = {"status_describe":"{}".format(error)}

        #回傳一個錯誤的描述
        return jsonify(result)	
    else:
        # 建查 user_id 是否存在
        if users['user_id'] == None:
            error = 'The user_id of user is None !'
            result = {"status_describe":"{}".format(error)}

        # 檢查 display_name 是否存在
        elif users['display_name'] == None:
            error = 'The display_name of user is None !'
            result = {"status_describe":"{}".format(error)}

        else:

            # 將使用者資料存入
            insertsql=("INSERT INTO chatbot_db.Users (user_id, display_name, picture_url, status_message, join_datetime) VALUES ( %s,%s,%s,%s,%s )") 
            value = (users['user_id'],
                     users['display_name'],
                     users['picture_url'],
                     users['status_message'],
                     join_datetime)

            cur.execute(insertsql , value)

            #將資料送進資料庫中
            conn.commit()

            # 傳回正確訊息 
            result =  { "status_describe":"success add user" }

    return jsonify(result)

In [6]:
# 取出所有id
@app.route('/users_id', methods= ['GET'])
def get_all_users():
    
    # 取出所有user_id
    cur.execute('SELECT user_id FROM chatbot_db.Users')

    # 取出多筆資料
    all_user_id = cur.fetchall()
    print('all_user_id: ',all_user_id)
    users = []
    for user in all_user_id:
        users.append(user[0])

    return jsonify(users)

In [7]:
#接口功能：檢視指定使用者資訊
#接口位置：/users/<userid>，運用了url parameter，使用get的http method
@app.route('/users/<user_id>',methods=['GET'])
#特別注意這邊有打userid，url parameter就是這樣使用
def read_user(user_id):
    #找出資料庫符合userid的資料
    cur.execute(
        'SELECT * FROM chatbot_db.Users WHERE user_id = ("%s")' % (user_id)
        )
    #將剛剛execute的資料取出來
    user = cur.fetchone()
    #假如有找到符合的資料，包裝成統一格式並回傳
    if user is not None:
        user = {
            "user_id":user[0],
            "display_name":user[1],
            "picture_url" : user[2],
            "status_message" : user[3],
            "join_datetime" : user[4],
        }
        #轉成line要的json格式
        return jsonify(user)
    #假如沒有找到符合的資料，回傳一個錯誤訊息
    else:
        result = {
            "status_describe":"Please enter the right id!!"
        }
        return jsonify(result)

In [8]:
#接口功能：檢視所有使用者資訊
#接口位置：/users，使用get的http method
@app.route('/users',methods=['GET'])
def read_users():
    #找出資料庫內的所有user資料
    cur.execute(
        'SELECT * FROM chatbot_db.Users'
        )
    #由於是多筆，使用fetchall
    user = cur.fetchall()
    #假如一個user都沒
    if not user:
        answer = {
          "status_describe":"query string is incompatible"
        }
    else:
        #裝成矩陣格式
        answer = []
        for i in user:
            result = {
                "user_id":i[0],
                "display_name":i[1],
                "picture_url" : i[2],
                "status_message" : i[3],
                "join_datetime" : i[4]
            }
            answer.append(result)
    
        
    #轉成json格式
    return jsonify(answer)

In [9]:
# 利用title取出新聞關鍵字
@app.route('/keyword_title/<title>', methods= ['Get'])
def get_keyword_by_title(title):
    
    cur.execute(
        'SELECT title_id FROM chatbot_db.News WHERE title = ("%s")' % (title)
    )
    title_id = cur.fetchone()
    answer = None
    if title_id == None:
        answer = {
            "This title : {} can't find" .format(title)
        }
    else:
        cur.execute(
            'SELECT keyword FROM chatbot_db.Article_Keyword JOIN chatbot_db.Keyword ON Article_Keyword.keyword_id = Keyword.keyword_id WHERE title_id = ("%s") ' % (title_id[0])
        )
        answer = []
        all_keyword = cur.fetchall()

        for k in all_keyword:
            answer.append(k[0])

    return jsonify(answer)

In [10]:
# 利用url, 取出所有關鍵字
@app.route('/keyword_url/', methods= ['POST'])
def get_keyword_by_url():
    
    news_url= request.get_json()
    url = news_url['url']
    cur.execute(
        'SELECT title_id FROM chatbot_db.News WHERE url = ("%s")' % (url)
    )
    title_id = cur.fetchone()
    cur.execute(
        'SELECT keyword FROM chatbot_db.Article_Keyword JOIN chatbot_db.Keyword ON Article_Keyword.keyword_id = Keyword.keyword_id WHERE title_id = ("%s") ' % (title_id[0])
    )
    all_keyword = cur.fetchall()
    keyword = []
    for k in all_keyword:
        keyword.append(k[0])

    return jsonify(keyword)

In [11]:
# 利用單個關鍵字，找出相關新聞。
@app.route('/keyword/<keyword>',methods=['GET'])
def get_keyword_title(keyword):
    
    cur.execute(
        'SELECT keyword_id FROM chatbot_db.Keyword WHERE keyword = ("%s")' % (keyword)
    )
    keyword_id = cur.fetchone()

    cur.execute(
        'SELECT title FROM chatbot_db.Article_Keyword JOIN chatbot_db.News ON News.title_id = Article_Keyword.title_id WHERE keyword_id = ("%s")' % (keyword_id[0])
    )
    all_title = cur.fetchall()
    news_tltle = []
    for title in all_title:
        news_tltle.append(title[0])
    
    return jsonify(news_tltle)

In [12]:
# 找出近7天的所有關鍵字並依照數量排序
@app.route('/seven_days_kwyword/',methods=['GET'])
def get_seven_days_kwywords():
    
    cur.execute (
        "select title_id from chatbot_db.News where date_sub(curdate(), INTERVAL 7 DAY) <= date(release_datetime)"
    )
    all_title= cur.fetchall()
    title_list = []
    for title in all_title:
        title_list.append(title[0])

    query = '''SELECT keyword , count(title_id) as num FROM chatbot_db.Article_Keyword JOIN chatbot_db.Keyword ON Article_Keyword.keyword_id = Keyword.keyword_id 
        WHERE title_id IN (%s{}) group by keyword order by num DESC'''.format(', %s' * (len(title_list)-1))

    cur.execute(
        query % (tuple(title_list))
    )

    all_keyword= cur.fetchall()    
    keyword_list = []
    
    for keyword in all_keyword:
        if not keyword[0] in ["國民黨", "民進黨"]: 
            keyword_list.append(keyword[0])
        
    return jsonify(keyword_list)

In [13]:


url = os.environ.get('GRAPHENEDB_URL', 'http://neo4j:7474')
 
graph = Graph(url + '/db/data/', username="neo4j", password="juce1234")

def create_titles():
    #Creates title nodes
    query = '''
    LOAD CSV WITH HEADERS FROM "file:///titles.csv" AS csvLine1  
    WITH csvLine1  
    WHERE csvLine1.title IS NOT NULL  
    MERGE (a_title:Title {title:csvLine1.title})  
    ON MATCH SET a_title.title = csvLine1.title 
    '''
    graph.run(query)
    
    
def create_keywords():
    query = '''
    LOAD CSV WITH HEADERS FROM "file:///keywords.csv" AS csvLine3
    FIELDTERMINATOR '|'
    WITH csvLine3  
    WHERE csvLine3.title IS NOT NULL and csvLine3.keyword IS NOT NULL 
    MERGE(keyword:Keyword{Keyword:csvLine3.keyword})  
    WITH csvLine3, keyword 
    MATCH(title:Title{title:csvLine3.title})                                                
    MERGE(title)-[r1:HAS_KEYWORDS]->(keyword)  
    '''
    graph.run(query)

def jaccard_kw_similarity():
    #After adding in new articles, run to find which articles are similar to each other. Jaccard score must be at least 0.1
    #create similarity relationship
    query = '''
    MATCH (search_query:Title)-[:HAS_KEYWORDS]->(k_id:Keyword)
    WITH {item:id(search_query), categories: collect(id(k_id))} as userData
    WITH collect(userData) as data
    CALL algo.similarity.jaccard(data, {topK: 1, similarityCutoff: 0.1, write:true})
    YIELD nodes, similarityPairs, write, writeRelationshipType, writeProperty, min, max, mean, stdDev, p25, p50, p75, p90, p95, p99, p999, p100
    RETURN nodes, similarityPairs, write, writeRelationshipType, writeProperty, min, max, mean, p95
    '''
    graph.run(query)
@app.route('/Keyword/<search>',methods=['GET'])
def jacc_kw_rec(search):
    q_r = []
    query2 = "MATCH (p:Title)-[:SIMILAR]->(other)-[:HAS_KEYWORDS]->(keyword1), (other)-[:HAS_KEYWORDS]->(keyword)  " \
             "where not((p)-[:HAS_KEYWORDS]->(keyword)) " \
             "and p.title contains \"{search}\"".format(search=search) + \
             " and keyword1 <> keyword RETURN DISTINCT keyword LIMIT 3"
 
    result = graph.run(query2)
 
    result = result.data()
    for r in result:
        r_j = json.dumps(r)
        loaded_r = json.loads(r_j)
        kw_ex = loaded_r['keyword']['Keyword']
        q_r.append(kw_ex)
        
    return jsonify(q_r)

@app.route('/news_g',methods=['GET'])        
def create_graph():
    create_titles()
    create_keywords()
    jaccard_kw_similarity()
    message = "All graphs have been created, have a nice day."
    return message
    


In [14]:
# 取出當日推播新聞
@app.route('/push_news',methods=['GET'])
def get_push_news():
    push_message_date = datetime.datetime.now().strftime("%Y-%m-%d")
    cur.execute('SELECT * FROM chatbot_db.PushMessage WHERE push_date = ("%s")' % (push_message_date))
    push_message= cur.fetchall()

    i = push_message[0]
    result = {
        "part_a1":i[1],
        "part_a1_url":i[2],
        "part_a2" : i[3],
        "part_a2_url" : i[4],
        "part_a3":i[5],
        "part_a3_url":i[6],
        "part_b1" : i[7],
        "part_b1_url" : i[8],
        "part_b2" : i[9],
        "part_b2_url" : i[10],
        "part_b3" : i[11],
        "part_b3_url" : i[12],
        "part_c1" : i[13],
        "part_c1_url" : i[14],
        "part_c2" : i[15],
        "part_c2_url" : i[16],
        "part_c3" : i[17],
        "part_c3_url" : i[18]       
    }

    return jsonify(result)


In [15]:
# 存入推播新聞
@app.route('/push_news',methods=['POST'])
def add_push_news():
    push_message_date = datetime.datetime.now().strftime("%Y-%m-%d")
    # 取得新聞資料
    push = request.get_json()
    
    cur.execute('SELECT part_a1_url FROM chatbot_db.PushMessage WHERE push_date = ("%s")' % (push_message_date))
    push_exit= cur.fetchone()
    if push_exit == None:
        
        insertsql=("INSERT INTO chatbot_db.PushMessage (push_date, part_a1, part_a1_url, part_a2, part_a2_url, part_a3, part_a3_url, part_b1, part_b1_url, part_b2, part_b2_url, part_b3, part_b3_url, part_c1, part_c1_url, part_c2, part_c2_url, part_c3, part_c3_url) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)") 
        value = (push_message_date,
                push['part_a1'],
                push['part_a1_url'],
                push['part_a2'],
                push['part_a2_url'],
                push['part_a3'],
                push['part_a3_url'],
                push['part_b1'],
                push['part_b1_url'],
                push['part_b2'],
                push['part_b2_url'],
                push['part_b3'],
                push['part_b3_url'],
                push['part_c1'],
                push['part_c1_url'],
                push['part_c2'],
                push['part_c2_url'],
                push['part_c3'],
                push['part_c3_url'],
        )

        cur.execute(insertsql , value)

        #將資料送進資料庫中
        conn.commit()
        result = {"status_describe":"push sucess send in mysql"}
    else:
        result = {"status_describe":"push_exit"}
    
    return jsonify(result)


In [16]:
# 利用title_id, 取出所有推播相關元素
@app.route('/push_needed/<title_id>', methods= ['GET'])
def get_push_need(title_id):
    
    cur.execute(
        'SELECT title, img_url, origin FROM chatbot_db.News JOIN chatbot_db.Origin ON News.origin_id = Origin.origin_id WHERE title_id = ("%s") ' % (title_id)
    )
    push = cur.fetchall()
    result={
    "title" : push[0][0],
    "img_url": push[0][1],
    "origin" : push[0][2]
    }
    
    return jsonify(result)

In [17]:
@app.route('/get_title_id/', methods= ['POST'])
def url_get_title_id():
    news_url= request.get_json()
    url = news_url['url']
    cur.execute(
    'SELECT title_id FROM chatbot_db.News WHERE url = ("%s")' % (url)
    )
    title_id= cur.fetchone()
    news= {"title_id": title_id[0]}
    return jsonify(news)

In [18]:
@app.route('/today_news',methods=['GET'])
def get_today_news():
    #找出資料庫內的所有user資料
    cur.execute(
        'select * from chatbot_db.News where Date(release_datetime) = curdate();'
        )
    #由於是多筆，使用fetchall
    all_news = cur.fetchall()
    #假如一個user都沒
    if not all_news:
        answer = {
          "status_describe":"query string is incompatible"
        }
    else:
        #裝成矩陣格式
        answer = []
        for i in all_news:
            result= {"title_id": i[0],
                    "origin_id": i[1], 
                    'title': i[4],
                    'url': i[5],
                    'content': i[7]
                   }
            answer.append(result)
    
        
    #轉成json格式
    return jsonify(answer)

In [19]:
@app.route('/get_elastic/', methods= ['POST'])
def article_search():
    text= request.get_json()
    text = text['text']
    def url_change_titleid(url):
        cur.execute(
        'SELECT title_id FROM chatbot_db.News WHERE url = ("%s")' % (url)
        )
        title_id= cur.fetchone()
        return title_id[0]
    
    body = {
            "query": {
              "match_phrase": {
                "content": {
                  "query": text,
                  "slop": 1
                }
              }
            }
    }
    all_hit_news =  es.search(index = "news", doc_type = "politics", body = body)
    news_list = []
    if len(all_hit_news['hits']['hits']) <= 10:
        for a in all_hit_news['hits']['hits']:
            news_list.append(url_change_titleid(a['_source']['url']))
    else:
        for p in range(10):
            news_list.append(url_change_titleid(all_hit_news['hits']['hits'][p]['_source']['url']))
    
    return jsonify(news_list)

In [20]:
import logging
#參考:http://zwindr.blogspot.com/2016/08/python-logging.html
# 基礎設定
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt='%m-%d %H:%M',
                    #製作名為my.log的檔案裝log
                    handlers = [logging.FileHandler('/home/jovyan/work/my.log', 'w', 'utf-8'),])
 
# 定義 handler 輸出 sys.stderr
console = logging.StreamHandler()
#定義要擷取的log最低等級到哪
console.setLevel(logging.DEBUG)
# 設定輸出格式
formatter = logging.Formatter('%(name)-12s: %(levelname)-8s %(message)s')
# handler 設定輸出格式
console.setFormatter(formatter)
# 加入 hander 到 root logger
logging.getLogger('').addHandler(console)

In [21]:
#__name__ == __main__ 代表你執行這個模塊（py檔）時會成立
#假如你是被別的檔案import的話，__name__ == 檔案名稱，這個if就不會成立
if __name__=='__main__':

    #運行flask server，運行在0.0.0.0:5000
    #要特別注意假如運行在127.0.0.1的話，會變成只有本機連的到，外網無法
    app.run(host='0.0.0.0',port=5000)

werkzeug    : INFO      * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
urllib3.connectionpool: DEBUG    Starting new HTTP connection (1): neo4j:7474
urllib3.connectionpool: DEBUG    http://neo4j:7474:None "POST /db/data/transaction/commit HTTP/1.1" 200 897
[2019-01-08 02:03:37,565] ERROR in app: Exception on /Keyword/"蔡英文" [GET]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.6/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.6/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/opt/conda/lib/python3.6/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_

werkzeug    : INFO     172.22.0.1 - - [08/Jan/2019 02:19:53] "GET /Keyword/%E8%94%A1%E8%8B%B1%E6%96%87 HTTP/1.1" 200 -
urllib3.connectionpool: DEBUG    http://neo4j:7474:None "POST /db/data/transaction/commit HTTP/1.1" 200 3803
werkzeug    : INFO     172.22.0.1 - - [08/Jan/2019 02:20:01] "GET /Keyword/%E9%80%99%E4%BA%9B HTTP/1.1" 200 -
urllib3.connectionpool: DEBUG    http://neo4j:7474:None "POST /db/data/transaction/commit HTTP/1.1" 200 3808
werkzeug    : INFO     172.22.0.1 - - [08/Jan/2019 02:20:03] "GET /Keyword/%E5%8F%B0%E7%81%A3 HTTP/1.1" 200 -
urllib3.connectionpool: DEBUG    http://neo4j:7474:None "POST /db/data/transaction/commit HTTP/1.1" 200 3808
werkzeug    : INFO     172.22.0.1 - - [08/Jan/2019 02:20:10] "GET /Keyword/%E6%9F%AF%E6%96%87 HTTP/1.1" 200 -
urllib3.connectionpool: DEBUG    http://neo4j:7474:None "POST /db/data/transaction/commit HTTP/1.1" 200 3814
werkzeug    : INFO     172.22.0.1 - - [08/Jan/2019 02:20:12] "GET /Keyword/%E7%B2%89%E7%B5%B2 HTTP/1.1" 200 -
urlli

werkzeug    : INFO     172.22.0.1 - - [08/Jan/2019 02:31:14] "POST /get_elastic/ HTTP/1.1" 200 -
urllib3.connectionpool: DEBUG    http://elasticsearch:9200 "GET /news/politics/_search HTTP/1.1" 200 25903
elasticsearch: INFO     GET http://elasticsearch:9200/news/politics/_search [status:200 request:0.003s]
elasticsearch: DEBUG    > {"query":{"match_phrase":{"content":{"query":"蔡英文","slop":1}}}}
elasticsearch: DEBUG    < {"took":0,"timed_out":false,"_shards":{"total":5,"successful":5,"skipped":0,"failed":0},"hits":{"total":423,"max_score":3.0422084,"hits":[{"_index":"news","_type":"politics","_id":"0d_TJ2gB8NdVyYv62tOx","_score":3.0422084,"_source":{"source":"風傳媒 THE STORM MEDIA","url":"https://www.storm.mg/article/794155","title":"3個月累積近159萬筆！蔡英文網路聲量起死回生　遠超韓國瑜、柯文哲","date_":"2019-01-06 12:48","author":["趙宥寧"],"content":"歷經九合一選舉大敗，總統蔡英文不僅辭去民進黨主席一職，聲勢更一落千丈。不過「網路溫度計」調查顯示，蔡英文近日卻在兩岸議題上爆紅，其強勢回應中國國家主席習近平「一國兩制」的決心，以及日前4位綠營獨派大老登報逼宮蔡英文等事件，順勢拉抬了蔡英文的網路聲量與好感度。強硬態度回應習近平 單日聲量超過6萬筆根據《KEYPO大數據關鍵引擎》的聲量趨勢

werkzeug    : INFO     172.22.0.1 - - [08/Jan/2019 02:36:38] "POST /get_elastic/ HTTP/1.1" 200 -
urllib3.connectionpool: DEBUG    http://elasticsearch:9200 "GET /news/politics/_search HTTP/1.1" 200 25903
elasticsearch: INFO     GET http://elasticsearch:9200/news/politics/_search [status:200 request:0.003s]
elasticsearch: DEBUG    > {"query":{"match_phrase":{"content":{"query":"蔡英文","slop":1}}}}
elasticsearch: DEBUG    < {"took":0,"timed_out":false,"_shards":{"total":5,"successful":5,"skipped":0,"failed":0},"hits":{"total":423,"max_score":3.0422084,"hits":[{"_index":"news","_type":"politics","_id":"0d_TJ2gB8NdVyYv62tOx","_score":3.0422084,"_source":{"source":"風傳媒 THE STORM MEDIA","url":"https://www.storm.mg/article/794155","title":"3個月累積近159萬筆！蔡英文網路聲量起死回生　遠超韓國瑜、柯文哲","date_":"2019-01-06 12:48","author":["趙宥寧"],"content":"歷經九合一選舉大敗，總統蔡英文不僅辭去民進黨主席一職，聲勢更一落千丈。不過「網路溫度計」調查顯示，蔡英文近日卻在兩岸議題上爆紅，其強勢回應中國國家主席習近平「一國兩制」的決心，以及日前4位綠營獨派大老登報逼宮蔡英文等事件，順勢拉抬了蔡英文的網路聲量與好感度。強硬態度回應習近平 單日聲量超過6萬筆根據《KEYPO大數據關鍵引擎》的聲量趨勢

werkzeug    : INFO     172.22.0.1 - - [08/Jan/2019 02:39:32] "POST /get_elastic/ HTTP/1.1" 200 -
urllib3.connectionpool: DEBUG    http://elasticsearch:9200 "GET /news/politics/_search HTTP/1.1" 200 25903
elasticsearch: INFO     GET http://elasticsearch:9200/news/politics/_search [status:200 request:0.003s]
elasticsearch: DEBUG    > {"query":{"match_phrase":{"content":{"query":"    蔡英文","slop":1}}}}
elasticsearch: DEBUG    < {"took":0,"timed_out":false,"_shards":{"total":5,"successful":5,"skipped":0,"failed":0},"hits":{"total":423,"max_score":3.0422084,"hits":[{"_index":"news","_type":"politics","_id":"0d_TJ2gB8NdVyYv62tOx","_score":3.0422084,"_source":{"source":"風傳媒 THE STORM MEDIA","url":"https://www.storm.mg/article/794155","title":"3個月累積近159萬筆！蔡英文網路聲量起死回生　遠超韓國瑜、柯文哲","date_":"2019-01-06 12:48","author":["趙宥寧"],"content":"歷經九合一選舉大敗，總統蔡英文不僅辭去民進黨主席一職，聲勢更一落千丈。不過「網路溫度計」調查顯示，蔡英文近日卻在兩岸議題上爆紅，其強勢回應中國國家主席習近平「一國兩制」的決心，以及日前4位綠營獨派大老登報逼宮蔡英文等事件，順勢拉抬了蔡英文的網路聲量與好感度。強硬態度回應習近平 單日聲量超過6萬筆根據《KEYPO大數據關鍵引擎》的

werkzeug    : INFO     172.22.0.1 - - [08/Jan/2019 02:45:36] "POST /get_elastic/ HTTP/1.1" 200 -
urllib3.connectionpool: DEBUG    http://elasticsearch:9200 "GET /news/politics/_search HTTP/1.1" 200 34313
elasticsearch: INFO     GET http://elasticsearch:9200/news/politics/_search [status:200 request:0.005s]
elasticsearch: DEBUG    > {"query":{"match_phrase":{"content":{"query":"    九二共識","slop":1}}}}
elasticsearch: DEBUG    < {"took":1,"timed_out":false,"_shards":{"total":5,"successful":5,"skipped":0,"failed":0},"hits":{"total":224,"max_score":8.056536,"hits":[{"_index":"news","_type":"politics","_id":"W9-5J2gB8NdVyYv6aMvP","_score":8.056536,"_source":{"source":"TVBS新聞台","url":"https://news.tvbs.com.tw/politics/1060755","title":"蔡要求別講九二共識 朱立倫重砲批扭曲","date_":"2019/01/06 22:39","author":["李品誼"],"content":"蔡總統日前要各政黨，不要再講九二共識，引發藍營反彈，其中就屬朱立倫砲火最猛烈，在臉書上批評蔡總統刻意扭曲九二共識的內涵，更無權要求大家不能提，不過就在習近平的談話後，大陸央視剛好播出朱立倫的訪問，內容談得就是九二共識。前國民黨主席朱立倫：「我們常說九二共識，是一個重要的一個基礎，重要的一個里程碑。」再次定調九二共識，對兩岸關係的重要性，在習近平發表談話後，大陸央視剛好播出

werkzeug    : INFO     172.22.0.1 - - [08/Jan/2019 02:46:20] "POST /get_elastic/ HTTP/1.1" 200 -
urllib3.connectionpool: DEBUG    http://elasticsearch:9200 "GET /news/politics/_search HTTP/1.1" 200 34313
elasticsearch: INFO     GET http://elasticsearch:9200/news/politics/_search [status:200 request:0.004s]
elasticsearch: DEBUG    > {"query":{"match_phrase":{"content":{"query":"    九二共識","slop":1}}}}
elasticsearch: DEBUG    < {"took":1,"timed_out":false,"_shards":{"total":5,"successful":5,"skipped":0,"failed":0},"hits":{"total":224,"max_score":8.056536,"hits":[{"_index":"news","_type":"politics","_id":"W9-5J2gB8NdVyYv6aMvP","_score":8.056536,"_source":{"source":"TVBS新聞台","url":"https://news.tvbs.com.tw/politics/1060755","title":"蔡要求別講九二共識 朱立倫重砲批扭曲","date_":"2019/01/06 22:39","author":["李品誼"],"content":"蔡總統日前要各政黨，不要再講九二共識，引發藍營反彈，其中就屬朱立倫砲火最猛烈，在臉書上批評蔡總統刻意扭曲九二共識的內涵，更無權要求大家不能提，不過就在習近平的談話後，大陸央視剛好播出朱立倫的訪問，內容談得就是九二共識。前國民黨主席朱立倫：「我們常說九二共識，是一個重要的一個基礎，重要的一個里程碑。」再次定調九二共識，對兩岸關係的重要性，在習近平發表談話後，大陸央視剛好播出

werkzeug    : INFO     172.22.0.1 - - [08/Jan/2019 02:46:52] "POST /get_elastic/ HTTP/1.1" 200 -
urllib3.connectionpool: DEBUG    http://elasticsearch:9200 "GET /news/politics/_search HTTP/1.1" 200 34313
elasticsearch: INFO     GET http://elasticsearch:9200/news/politics/_search [status:200 request:0.003s]
elasticsearch: DEBUG    > {"query":{"match_phrase":{"content":{"query":"    九二共識","slop":1}}}}
elasticsearch: DEBUG    < {"took":0,"timed_out":false,"_shards":{"total":5,"successful":5,"skipped":0,"failed":0},"hits":{"total":224,"max_score":8.056536,"hits":[{"_index":"news","_type":"politics","_id":"W9-5J2gB8NdVyYv6aMvP","_score":8.056536,"_source":{"source":"TVBS新聞台","url":"https://news.tvbs.com.tw/politics/1060755","title":"蔡要求別講九二共識 朱立倫重砲批扭曲","date_":"2019/01/06 22:39","author":["李品誼"],"content":"蔡總統日前要各政黨，不要再講九二共識，引發藍營反彈，其中就屬朱立倫砲火最猛烈，在臉書上批評蔡總統刻意扭曲九二共識的內涵，更無權要求大家不能提，不過就在習近平的談話後，大陸央視剛好播出朱立倫的訪問，內容談得就是九二共識。前國民黨主席朱立倫：「我們常說九二共識，是一個重要的一個基礎，重要的一個里程碑。」再次定調九二共識，對兩岸關係的重要性，在習近平發表談話後，大陸央視剛好播出

werkzeug    : INFO     172.22.0.1 - - [08/Jan/2019 02:47:57] "POST /get_elastic/ HTTP/1.1" 200 -
